<img src="https://afs-services.com/wp-content/uploads/2025/09/pypricing_banner.png" alt="RV Banner" style="width: 100%; height: auto; display: block;">

#  Importing

general packages

In [1]:
import sys, os, json
import numpy as np
import pandas as pd

our library

In [2]:
repo_root = os.path.abspath("..")
sys.path.insert(0, repo_root)
import afslibrary as afs

**importing data objects**

We first need a "database translator" like Beautiful Data, which transforms database queries into python synthax:

In [3]:
import db_tools

db = db_tools.BeautifulDataAFSStyle()
d = afs.DataFactory(db)

we can now import the data objects

In [4]:
calendars = d.import_calendar("Act360", "Act365", "Cal30360")
underlyings = d.import_underlying(
    "SX5E",
    "RTY",
    "ESTRON Index",
    start_date="20200101",
    end_date="20211231",
)

# American/Bermudan Options

## DLIB Example

Example p.9 of American Monte Carlo DLIB


In [5]:
calendar = "Act360"
paths = np.array(
    [
        [8.0, 11.0, 15.0, 18.0],
        [8.0, 10.0, 11.0, 6.0],
        [8.0, 9.0, 6.0, 6.0],
        [8.0, 6.0, 5.0, 8.0],
        [8.0, 4.0, 3.0, 2.0],
    ]
)
paths = paths[:, :, np.newaxis, np.newaxis]
curve = afs.CRDC(0.0, calendars[calendar])
strike = 9
dates = ["20200202"]
pay_dates = afs.dates_formatting(["20200202", "20240201", "20250201", "20260201"])
dates_dic = {}
for i in range(pay_dates.size):
    dates_dic[pay_dates[i]] = i
simulation_data = [[afs.dates_formatting(pay_dates[0]), paths, 0, dates_dic]]
american_put = afs.AmericanVanillaOption(
    underlying=underlyings["SX5E"],
    strike=strike,
    kind="put",
    calendar=calendars[calendar],
    pay_dates=pay_dates,
    nominal=1,
)
american_call = afs.AmericanVanillaOption(
    underlying=underlyings["SX5E"],
    strike=strike,
    kind="call",
    calendar=calendars[calendar],
    pay_dates=pay_dates,
    nominal=1,
)
european_call = afs.Call(
    underlying=underlyings["SX5E"],
    strike=strike,
    calendar=calendars[calendar],
    maturity=pay_dates[-1],
    nominal=1 * strike,
)
european_put = afs.Put(
    underlying=underlyings["SX5E"],
    strike=strike,
    calendar=calendars[calendar],
    maturity=pay_dates[-1],
    nominal=1 * strike,
)
engine = afs.RegressionMC()

Compute prices

In [6]:
price_data = {
    "Option Type": [
        "American Put (LS Method)",
        "American Put (TvR Method)",
        "European Put (MC Simulations)",
        "American Call (LS Method)",
        "American Call (TvR Method)",
        "European Call (MC Simulations)",
    ],
    "Price": [
        engine.compute_price_from_simulations(
            american_put, curve, simulation_data, method=("LS", 1)
        ).values[0],
        engine.compute_price_from_simulations(
            american_put, curve, simulation_data, method=("TvR", 3)
        ).values[0],
        afs.DeterministicVolDiffusionMC()
        .compute_price_from_simulations(european_put, curve, simulation_data)
        .values[0],
        engine.compute_price_from_simulations(
            american_call, curve, simulation_data, method=("LS", 3)
        ).values[0],
        engine.compute_price_from_simulations(
            american_call, curve, simulation_data, method=("TvR", 3)
        ).values[0],
        afs.DeterministicVolDiffusionMC()
        .compute_price_from_simulations(european_call, curve, simulation_data)
        .values[0],
    ],
}
df = pd.DataFrame(price_data)
df

,Option Type,Price
0,American Put (LS Method),3.2
1,American Put (TvR Method),3.4
2,European Put (MC Simulations),25.2
3,American Call (LS Method),2.2
4,American Call (TvR Method),2.2
5,European Call (MC Simulations),16.2


## Glasserman

Example of Glasserman, Chapter 8, p.463

In [7]:
# Data
calendar = "Act360"
curve = afs.CRDC(0.05, calendars[calendar])
strike = 100
dates = ["20200203"]
pay_dates = pd.DatetimeIndex(
    [
        pd.to_datetime(date) + pd.DateOffset(days=i * 360 / 3)
        for date in dates
        for i in range(1, 10)
    ]
)

data = pd.DataFrame(
    index=underlyings["SX5E"].get_dates().union(underlyings["RTY"].get_dates()),
    columns=["Price", "Volatility", "Dividend Rate"],
)
data["Price"] = 100
data["Volatility"] = 0.2
data["Dividend Rate"] = 0.1
underlyings["SX5E"].set_data(data)
underlyings["RTY"].set_data(data)
corr = {}
for date in dates:
    corr[pd.to_datetime(date)] = np.array([[1, 0], [0, 1]])
underlyings_multi = afs.MultiAsset(underlyings["SX5E"], underlyings["RTY"])
underlyings_multi.corr = corr
underlyings_single = underlyings["SX5E"]

Products

In [8]:
american_put = afs.AmericanVanillaOption(
    underlying=underlyings_multi,
    strike=strike,
    kind="put",
    calendar=calendars[calendar],
    pay_dates=pay_dates,
    nominal=1,
    phi="max",
)
american_call = afs.AmericanVanillaOption(
    underlying=underlyings_multi,
    strike=strike,
    kind="call",
    calendar=calendars[calendar],
    pay_dates=pay_dates,
    nominal=1,
    phi="max",
)
european_call = afs.Call(
    underlying=underlyings_multi,
    maturity=pay_dates[-1],
    strike=strike,
    calendar=calendars[calendar],
    nominal=strike,
    phi="max",
)
# Nominal to make it comparable

Monte Carlo Pricing

In [9]:
engine_am = afs.RegressionMC()
no_paths = 10**6

In [10]:
# Create a dataframe to collect results
results_df = pd.DataFrame(columns=["Price", "Method", "Actual", "Expected"])
prices = np.array([90, 100, 110])
methods = [("LS", 2), ("TvR", 2)]
expected = {
    (90, ("TvR", 2)): 8.27,
    (90, ("LS", 2)): 7.99,
    (100, ("TvR", 2)): 14.08,
    (100, ("LS", 2)): 13.78,
    (110, ("TvR", 2)): 21.38,
    (110, ("LS", 2)): 21.16,
}

# Collecting values in the loop to store in the dataframe
for price in prices:
    for method in methods:
        data["Price"] = price
        underlyings["SX5E"].set_data(data)
        underlyings["RTY"].set_data(data)
        actual_value = engine_am.price(
            american_call, dates, curve, no_paths, method=method
        )
        expected_value = expected[(price, method)]

        # Append to dataframe using concatenate
        new_row = pd.DataFrame(
            {
                "Price": [price],
                "Method": [str(method)],
                "Actual": [actual_value.values[0]],
                "Expected": [expected_value],
                "Realative error": [
                    (actual_value.values[0] - expected_value) / expected_value
                ],
            }
        )
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display dataframe to get the table
results_df

,Price,Method,Actual,Expected,Realative error
0,90,"('LS', 2)",8.024131,7.99,0.004272
1,90,"('TvR', 2)",8.201798,8.27,-0.008247
2,100,"('LS', 2)",13.834669,13.78,0.003967
3,100,"('TvR', 2)",13.993064,14.08,-0.006174
4,110,"('LS', 2)",21.229316,21.16,0.003276
5,110,"('TvR', 2)",21.339791,21.38,-0.001881


## Longstaff-Schwartz paper

Data

In [11]:
calendar = "Act365"
no_paths = 10**5
curve = afs.CRDC(0.06, calendars[calendar])
strike = 40
date = "20200203"
date = pd.to_datetime(date)
T = 1
# pay_dates = pd.date_range(start=date, end=date+pd.DateOffset(years=T), periods=50*T+1, inclusive="right")
pay_dates = pd.date_range(start=date, end=date + pd.DateOffset(years=T), periods=50 * T)
data = pd.DataFrame(
    index=underlyings["SX5E"].get_dates().union(pay_dates),
    columns=["Price", "Volatility", "Dividend Rate"],
)
data["Price"] = 36
data["Volatility"] = 0.4
data["Dividend Rate"] = 0
underlyings["SX5E"].set_data(data)
underlyings_single = underlyings["SX5E"]

![alt text](Table1_LS.png)

Prepare a DataFrame to collect the results for put, as in Table 1 of the paper

In [12]:
results_df = pd.DataFrame(columns=["Type", "Method", "No. of Polynomials", "Price"])

# Put
european_put = afs.Put(
    underlying=underlyings_single,
    maturity=pay_dates[-1],
    strike=strike,
    calendar=calendars[calendar],
    nominal=strike,
)
results_df = pd.concat(
    [
        results_df,
        pd.DataFrame(
            {
                "Type": ["European Put"],
                "Method": ["Analytical"],
                "No. of Polynomials": [None],
                "Price": [european_put.get_px(date, curve).values[0]],
            }
        ),
    ]
)

for i in range(10):
    method = ("LS", i)
    engine_am = afs.RegressionMC()
    american_put = afs.AmericanVanillaOption(
        underlying=underlyings_single,
        strike=strike,
        kind="put",
        calendar=calendars[calendar],
        pay_dates=pay_dates,
        nominal=1,
    )

    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "Type": ["American Put", "American Put TvR"],
                    "Method": [f"LS (no.poly={i})", f"TvR (no.poly={i})"],
                    "No. of Polynomials": [i, i],
                    "Price": [
                        engine_am.price(
                            american_put,
                            date,
                            curve,
                            no_paths,
                            method=method,
                            no_calcs=10,
                        ).values[0],
                        engine_am.price(
                            american_put,
                            date,
                            curve,
                            no_paths,
                            method=("TvR", method[1]),
                            no_calcs=10,
                        ).values[0],
                    ],
                }
            ),
        ],
        ignore_index=True,
    )

results_df


,Type,Method,No. of Polynomials,Price
0,European Put,Analytical,None,268.615049
1,American Put,LS (no.poly=0),0,7.032711
2,American Put TvR,TvR (no.poly=0),0,7.373926
3,American Put,LS (no.poly=1),1,7.005352
4,American Put TvR,TvR (no.poly=1),1,7.474750
5,American Put,LS (no.poly=2),2,7.035565
6,American Put TvR,TvR (no.poly=2),2,7.259721
7,American Put,LS (no.poly=3),3,7.071566
8,American Put TvR,TvR (no.poly=3),3,7.236179
9,American Put,LS (no.poly=4),4,7.084458


The last prices, with a lot of polynomials are completely off. This is due to the presumably high oscillations to fix the data. 
Now the same to check that the price of the american call has no premium.

In [13]:
data["Price"] = 40
data["Volatility"] = 0.4
data["Dividend Rate"] = 0
underlyings["SX5E"].set_data(data)
underlyings_single = underlyings["SX5E"]

european_call = afs.Call(
    underlying=underlyings_single,
    maturity=pay_dates[-1],
    strike=strike,
    calendar=calendars[calendar],
    nominal=strike,
)
results_df = pd.DataFrame(
    {
        "Type": ["European Call"],
        "Method": ["Analytical"],
        "No. of Polynomials": [None],
        "Price": [european_call.get_px(date, curve).values[0]],
    }
)

for i in range(10):
    method = ("LS", i)
    engine_am = afs.RegressionMC()
    american_call = afs.AmericanVanillaOption(
        underlying=underlyings_single,
        strike=strike,
        kind="call",
        calendar=calendars[calendar],
        pay_dates=pay_dates,
        nominal=1,
    )

    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                {
                    "Type": ["American Call", "American Call TvR"],
                    "Method": [f"LS (no.poly={i})", f"TvR (no.poly={i})"],
                    "No. of Polynomials": [i, i],
                    "Price": [
                        engine_am.price(
                            american_call,
                            date,
                            curve,
                            no_paths,
                            method=method,
                            no_calcs=10,
                        ).values[0],
                        engine_am.price(
                            american_call,
                            date,
                            curve,
                            no_paths,
                            method=("TvR", method[1]),
                            no_calcs=10,
                        ).values[0],
                    ],
                }
            ),
        ],
        ignore_index=True,
    )

# Display the results
results_df

,Type,Method,No. of Polynomials,Price
0,European Call,Analytical,None,296.009293
1,American Call,LS (no.poly=0),0,7.404530
2,American Call TvR,TvR (no.poly=0),0,7.413425
3,American Call,LS (no.poly=1),1,7.267327
4,American Call TvR,TvR (no.poly=1),1,7.707567
5,American Call,LS (no.poly=2),2,7.319353
6,American Call TvR,TvR (no.poly=2),2,7.611647
7,American Call,LS (no.poly=3),3,7.311927
8,American Call TvR,TvR (no.poly=3),3,7.509249
9,American Call,LS (no.poly=4),4,7.360781


# Bermudan products

First, European product:

In [14]:
offset = 10
tenor = 1
strike = -0.005
date_alpha = date + pd.DateOffset(days=offset)
date_1 = date_alpha + pd.DateOffset(months=6)
date_beta = date_alpha + pd.DateOffset(years=tenor)
swap_rate = afs.LognormalSwapRate(
    curve,
    date_alpha,
    date_beta,
    6,
    6,
    legs_calendars=calendars["Act360"],
    tenor_length=tenor,
)
eur_swaption = afs.PayersSwaption(swap_rate, strike, calendars["Act360"], nominal=1)


American product

In [15]:
pay_dates = pd.date_range(start=date_alpha, end=date_beta, freq=pd.DateOffset(months=6))
bermudan_swaption = afs.AmericanFromEuropean(
    swap_rate, pay_dates, calendars["Act360"], eur_swaption
)

Interest rate model


In [16]:
date = pd.to_datetime("20200131")
dates = pay_dates.union([date])
curve = afs.CRDC(0.05, calendars["Act360"])
date_index = pd.DatetimeIndex(dates)
g2pp = afs.G2PlusPlusShortRate(curve, calendars["Act360"])
g2pp.parameters = g2pp.parameters.reindex(date_index)
a = 0.00116
b = 0.083
sigma = 0.00108
eta = 0.00102
rho = -0.5
g2pp.parameters["vol_x"] = sigma
g2pp.parameters["reversion_x"] = a
g2pp.parameters["vol_y"] = eta
g2pp.parameters["reversion_y"] = b
g2pp.parameters["correlation"] = rho

In [17]:
mc_eur = afs.SRDeterministicVolDiffusionMC(g2pp)
mc_eur.price(eur_swaption, date, curve, 1000000)

2020-01-31    0.054359
dtype: float64

In [18]:
mc = afs.SRRegressionMC(g2pp)
mc.price(bermudan_swaption, date, curve, 1000000, method=("LS", 2))

2020-01-31    0.163074
dtype: float64

In [19]:
mc.compute_disc_payoff_from_simulations

<bound method DiffusionMC.compute_disc_payoff_from_simulations of <pricing.mc_engines.SRRegressionMC object at 0x000002E0C8F04890>>